## Introducción a la Ciencia de Datos con Julia

### Primeros pasos:
- Importación de los datos

In [4]:
# TODO: Activamos nuestros paquetes 
using CSV, DataFrames, DataFramesMeta, ArchGDAL, PrettyTables

In [25]:
# TODO: Importación y creación de un DataFrame desde un archivo CSV

df_pop1 = DataFrame(CSV.File("data/table_pop_huanuco.csv"))

first(df_pop1,10)

,fid,population,full_id,name,pe_ubigeo
,Int64,Int64,String?,String?,Int64?
1,2,196,r1954518,Cholón,100702
2,3,344,r1954518,Cholón,100702
3,7,76,r1954518,Cholón,100702
4,10,44,r1954518,Cholón,100702
5,11,120,r1954518,Cholón,100702
6,12,32,r1954518,Cholón,100702
7,13,44,r1954518,Cholón,100702
8,14,66,r1954518,Cholón,100702
9,15,213,r1954518,Cholón,100702


In [52]:
# TODO: Importación y creación de un DataFrame desde un archivo GeoJSON

dat_geo = ArchGDAL.read("data/select_pop_huanuco.geojson")
layer_geo = ArchGDAL.getlayer(dat_geo, 0)
df_pop2 = DataFrames.DataFrame(layer_geo)

first(df_pop2, 10)


,,fid,population,full_id,name,pe_ubigeo
,IGeomet…,Float64,Float64,String,String,String
1,Geometry: wkbPoint,2.0,196.0,r1954518,Cholón,100702
2,Geometry: wkbPoint,3.0,344.0,r1954518,Cholón,100702
3,Geometry: wkbPoint,7.0,76.0,r1954518,Cholón,100702
4,Geometry: wkbPoint,10.0,44.0,r1954518,Cholón,100702
5,Geometry: wkbPoint,11.0,120.0,r1954518,Cholón,100702
6,Geometry: wkbPoint,12.0,32.0,r1954518,Cholón,100702
7,Geometry: wkbPoint,13.0,44.0,r1954518,Cholón,100702
8,Geometry: wkbPoint,14.0,66.0,r1954518,Cholón,100702
9,Geometry: wkbPoint,15.0,213.0,r1954518,Cholón,100702


In [56]:
typeof(dat_geo)

ArchGDAL.IDataset

In [60]:
typeof(layer_geo)

ArchGDAL.IFeatureLayer

In [61]:
ArchGDAL.nfeature(layer_geo)

3708

In [26]:
describe(df_pop1) #? Verificamos si existen datos faltantes

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,fid,3485.84,2,3192.5,8219,0,Int64
2,population,213.863,25,82.0,9209,0,Int64
3,full_id,,r1954505,,r7899608,2,"Union{Missing, String}"
4,name,,Amarilis,,Yuyapichis,2,"Union{Missing, String}"
5,pe_ubigeo,1.00515e5,100101,100507.0,101108,2,"Union{Missing, Int64}"


In [27]:
# TODO: Convertir un DataFrame a formato HTML
# ! Empleamos las bondades del paquete PrettyTables (https://ronisbr.github.io/PrettyTables.jl/stable/)

function savehtml(filename, data)
    open("$filename.html", "w") do f
        pretty_table(f, data, backend=:html, tf=tf_html_dark)
    end
end

savehtml("html/pop_huanuco_points", df_pop1)

In [28]:
# TODO: Ordenar los datos por nombre

df_pop1 = sort!(df_pop1, :name);
savehtml("html/pop_huanuco_points_ord", df_pop1)


In [29]:
# TODO: Eliminar datos faltantes

df_pop1 = dropmissing!(df_pop1);
savehtml("html/pop_huanuco_points_ajust", df_pop1)
describe(df_pop1)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,fid,3484.53,2,3191.5,8219,0,Int64
2,population,213.938,25,82.0,9209,0,Int64
3,full_id,,r1954505,,r7899608,0,String
4,name,,Amarilis,,Yuyapichis,0,String
5,pe_ubigeo,1.00515e5,100101,100507.0,101108,0,Int64


In [38]:
# TODO: Agrupar y combinar columnas de datos
#* Agrupamos los valores por los nombres de los distritos empleando "groupby"
df_gpop = groupby(df_pop1, :name);
#* Combinamos los resultados incorporando nuevos campos empleando "combine"
df_gpop_c = combine(df_gpop,
    :population => sum,
    :pe_ubigeo => unique
    );
savehtml("html/pop_huanuco_agcomb", df_gpop_c)


In [40]:
# TODO: Crear una columna índice

df_gpop_c.id = 1:nrow(df_gpop_c);  #* creación de una columna "id"
df_gpop_c = select!(df_gpop_c, :id, :); #* modificar de manera permanente nuestro dataframe
savehtml("html/pob_huanuco_index", df_gpop_c)

In [47]:
# TODO: Modificar los encabezados

rename!(df_gpop_c, [:id, :nombre, :población, :ubigeo])
savehtml("html/pob_huanuco_new", df_gpop_c)


In [48]:
# TODO: Convertir Int64 a String
#* La referencia a columnas en el nuevo objeto se obtiene usando "!" para la posición de la fila (para enfatizar que los datos referenciados podrían cambiarse en el nuevo objeto).

df_gpop_c[!, :ubigeo] = string.(df_gpop_c[:, :ubigeo]) #! Ojo: df[!, :col] es lo mismo que df.col 
savehtml("html/pob_huanuco_convert", df_gpop_c)

# Otra opción es: df[:, :col] .= value

In [50]:
# TODO: Guardar nuestros resultados como CSV

CSV.write("data/pop_dist_huanuco.csv", df_gpop_c)

"data/pop_dist_huanuco.csv"

In [64]:
#* Generar nuevos DataFrames empleando filtros

df_gpop_cmax = @subset(df_gpop_c, :población .> 5000) #! El uso de macros es gracias al paquete DataFramesMeta
savehtml("html/pob_huanuco_max", df_gpop_cmax)

In [67]:
df_gpop_ord = @linq df_gpop_c |>
                        where(:población .> 10000) |>
                        orderby(:ubigeo)
savehtml("html/pob_huanuco_ord", df_gpop_ord)